# Binance-Lighter Arbitrage Dry-Run Test
Use this notebook to verify connectivity and logic before running the main bot script.

In [ ]:
import os
import sys
import asyncio
import yaml

# Add src to path
sys.path.append(os.path.abspath('src'))

from utils.config_loader import load_config
from exchanges.binance_client import BinanceClientWrapper
from exchanges.lighter_client import LighterClientWrapper
from core.strategy import ArbStrategy
from core.engine import TradeEngine

print("Modules imported successfully")

## 1. Load Configuration

In [ ]:
config = load_config("config/settings.yaml")
print("Config loaded for Binance:", config['strategy']['symbol_binance'])
print("Config loaded for Lighter:", config['strategy']['symbol_lighter'])

## 2. Test Strategy Logic
Verify that the maker prices calculated for Binance make sense given Lighter's taker prices.

In [ ]:
strategy = ArbStrategy(
    min_profit_pct=config['strategy']['min_profit_pct'],
    binance_fee_pct=config['strategy']['binance_fee_pct'],
    lighter_fee_pct=config['strategy']['lighter_fee_pct']
)

# Manual Orderbook Mock (Price, Quantity)
lighter_best_bid = 50000.0
lighter_best_ask = 50010.0

binance_maker_prices = strategy.calculate_binance_maker_prices(lighter_best_bid, lighter_best_ask)

print(f"Lighter Taker Bid: {lighter_best_bid} | Ask: {lighter_best_ask}")
print(f"Binance Maker Bid: {binance_maker_prices['bid']:.2f} (We buy on Binance, later sell on Lighter @ {lighter_best_bid})")
print(f"Binance Maker Ask: {binance_maker_prices['ask']:.2f} (We sell on Binance, later buy on Lighter @ {lighter_best_ask})")

## 3. Test Exchange Connectivity (Dry Run)
Wait for connection and fetch orderbooks.

In [ ]:
async def test_connectivity():
    binance = BinanceClientWrapper(
        api_key=config['exchanges']['binance']['api_key'],
        api_secret=config['exchanges']['binance']['api_secret'],
        testnet=config['exchanges']['binance']['testnet']
    )
    
    lighter = LighterClientWrapper(
        private_key=config['exchanges']['lighter']['private_key'],
        api_url=config['exchanges']['lighter']['api_url'],
        web3_url=config['exchanges']['lighter']['web3_url']
    )
    
    print("Connecting to Binance...")
    await binance.connect()
    
    print("Connecting to Lighter...")
    await lighter.connect()
    
    # Fetch Orderbooks
    print(f"Fetching Binance {config['strategy']['symbol_binance']}...")
    b_ob = await binance.get_orderbook(config['strategy']['symbol_binance'])
    
    print(f"Fetching Lighter {config['strategy']['symbol_lighter']}...")
    l_ob = await lighter.get_orderbook(config['strategy']['symbol_lighter'])
    
    print(f"Binance Best Bid: {b_ob['bids'][0][0]}")
    print(f"Lighter Orderbook retrieved successfully.")
    
    await binance.disconnect()
    await lighter.disconnect()

await test_connectivity()

## 4. Simulate Hedge Order Generation
This simulates what happens when Binance notifies us of a fill.

In [ ]:
mock_fill_msg = {
    'e': 'executionReport',
    'S': 'BUY',       # We bought on Binance
    'l': '0.001',     # Quantity
    'L': '50000.0',   # Price
    'x': 'TRADE'
}

hedge_params = strategy.get_hedge_order_details(mock_fill_msg)
print(f"Simulated Binance Fill: {mock_fill_msg['S']} {mock_fill_msg['l']} @ {mock_fill_msg['L']}")
print(f"Generated Lighter Hedge: {hedge_params['side']} {hedge_params['quantity']}")